In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
from sklearn.impute import KNNImputer
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
import catboost
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
RANDOM_SEED = 42

In [2]:
target = 'contest-tmp2m-14d__tmp2m'

In [3]:
def null_df(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).reset_index().rename(columns ={'index':'Name'})


### Reference: [load data (reduce memory usage) by GUILLAUME MARTIN](https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook)

In [4]:
def optimize_mem(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Load datasets and optimize memory

In [5]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
pd.set_option('display.max_column', None)

In [6]:
optimize_mem(train)
optimize_mem(test)

Mem. usage decreased to 352.24 Mb (50.1% reduction)
Mem. usage decreased to 29.48 Mb (49.7% reduction)


,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010-5,wind-uwnd-925-2010-6,w

## Number of unique in each columns

In [7]:
def get_nunique_df(df):
    nunique_df = df.nunique().reset_index().rename(columns={
        'index': 'Name',
        0: 'NumberOfUnique'
    }).sort_values(by = 'NumberOfUnique', ascending = False)
    return nunique_df

In [8]:
nunique_train = get_nunique_df(train)
#nunique_train.to_csv('number_unique.csv')

In [9]:
nunique_train

,Name,NumberOfUnique
0,index,375734
58,contest-tmp2m-14d__tmp2m,373563
71,contest-pres-sfc-gauss-14d__pres,346455
59,contest-slp-14d__slp,138455
15,contest-wind-h10-14d__wind-hgt-10,124346
...,...,...
1,lat,23
99,climateregions__climateregion,15
144,mei__meirank,14
141,mjo1d__phase,8


## Percentage of null value in each column
## Handle by filling null with average of prev and next available value

In [10]:
def null_count(df):
    return sum(df.isnull().sum())

In [11]:
def fill_nan_by_mean_prev_next(df, col):
    df[col].interpolate(inplace=True)

In [12]:
null_df = null_df(train)
null_df

,Name,Total,Percent
0,nmme0-prate-34w__ccsm30,15934,4.240766
1,nmme0-tmp2m-34w__ccsm30,15934,4.240766
2,ccsm30,15934,4.240766
3,nmme0-prate-56w__ccsm30,15934,4.240766
4,nmme-tmp2m-56w__ccsm3,10280,2.735978
...,...,...,...
241,cancm40,0,0.000000
242,ccsm40,0,0.000000
243,cfsv20,0,0.000000
244,gfdlflora0,0,0.000000


In [13]:
null_count(test)

0

In [14]:
null_list = null_df.loc[null_df['Percent'] > 0, 'Name']
null_list

0    nmme0-prate-34w__ccsm30
1    nmme0-tmp2m-34w__ccsm30
2                     ccsm30
3    nmme0-prate-56w__ccsm30
4      nmme-tmp2m-56w__ccsm3
5      nmme-prate-56w__ccsm3
6      nmme-tmp2m-34w__ccsm3
7      nmme-prate-34w__ccsm3
Name: Name, dtype: object

In [15]:
def fill_nan_by_mean_prev_next(df, col):
    df[col].interpolate(inplace=True)

In [16]:
for col in null_list:
    fill_nan_by_mean_prev_next(train, col)

In [17]:
null_count(train)

0

## Drop duplicates (if there is any)

In [18]:
#train.drop_duplicates(inplace=True)
#test.drop_duplicates(inplace=True)

## Feature engineer: get day month year columns

In [19]:
def feature_engineer(df):
    df['startdate'] = pd.to_datetime(train['startdate'])
    df['day'] = train.startdate.dt.day
    df['month'] = train.startdate.dt.month
    df['year'] = train.startdate.dt.year
#    df['month_year'] = train['startdate'].dt.to_period('M')

In [20]:
feature_engineer(train)

In [21]:
feature_engineer(test)

In [22]:
train['startdate'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 375734 entries, 0 to 375733
Series name: startdate
Non-Null Count   Dtype         
--------------   -----         
375734 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.9 MB


In [23]:
train.head(15)

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

## Location analysis
## We analysed that longtitude column in the train dataset differ to the test dataset in the 16th digit after the after the decimal
## Solution: round up the longtitude column values to the 14th digit
## Reference: [WiDS 2023: different locations train/test SOLVED by FLAVIA FELICIONI](https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved)

In [24]:
def scale_feature_location(train, test):
    scale = 14
    train.loc[:, 'lat'] = round(train.lat, scale)
    train.loc[:, 'lon'] = round(train.lon, scale)
    test.loc[:, 'lat'] = round(test.lat, scale)
    test.loc[:, 'lon'] = round(test.lon, scale)

    train_and_test = pd.concat([train, test], axis=0)
    train_and_test['loc_group'] = train_and_test.groupby(['lat',
                                                          'lon']).ngroup()
    print(f'{train_and_test.loc_group.nunique()} unique locations')

    train = train_and_test.iloc[:len(train)]
    test = train_and_test.iloc[len(train):].drop(target, axis=1)

    return train, test

In [25]:
train, test = scale_feature_location(train, test)

514 unique locations


In [26]:
def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

In [27]:
def get_correlation_to_target(data, target_col):
    corr = data.corr()[target_col].sort_values(ascending=False)
    return corr

In [28]:
#target_feature_corr = get_correlation_to_target(train, target).to_frame()

In [29]:
#abs(target_feature_corr).nsmallest(columns = 'contest-tmp2m-14d__tmp2m', n = 50)

In [30]:
train

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

## Modeling

In [31]:
drop_useless_col = ['index', 'startdate' ,'climateregions__climateregion']

In [32]:
train_dropped = train.drop(columns = drop_useless_col)
test_dropped = test.drop(columns = drop_useless_col)

In [33]:
X, y = train_dropped.drop(columns=target), train_dropped[target]
print(any(target in target for target in X.columns))
print(y.name)

True
contest-tmp2m-14d__tmp2m


In [51]:
lasso = Lasso(alpha=1).fit(X, y)
model = SelectFromModel(lasso, prefit=True)
X_new = model.transform(X)

# Print the number of features before and after feature selection
print("Number of features before:", X.shape[1])
print("Number of features after:", X_new.shape[1])

Number of features before: 246
Number of features after: 80


In [52]:
lasso_selected_col = X.columns[abs(lasso.coef_) > 0].values

In [53]:
model_df = X[lasso_selected_col]
model_df

,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__nasa0,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__gfdlflorb,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cfsv2,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__nasa0,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__nasa0,contest-slp-14d__slp,nmme-prate-56w__cancm3,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdlflorb,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,contest-wind-h850-14d__wind-hgt-850,contest-wind-h500-14d__wind-hgt-500,gfdlflora0,elevation__elevation,wind-vwnd-250-2010-4,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-17,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-10,wind-hgt-10-2010-1,wind-hgt-10-2010-2,wind-hgt-10-2010-3,wind-hgt-10-2010-4,wind-hgt-10-2010-5,wind-hgt-10-2010-6,wind-hgt-10-2010-7,wind-hgt-10-2010-8,wind-hgt-10-2010-9,wind-hgt-10-2010-10,wind-hgt-100-2010-1,wind-hgt-100-2010-2,wind-hgt-100-2010-3,wind-hgt-100-2010-4,wind-hgt-100-2010-5,wind-hgt-100-2010-7,wind-hgt-100-2010-8,wind-hgt-100-2010-9,loc_group
0,237.000000,29.66,31246.630859,28.719999,81.720001,38.049999,14.37,11.60,17.629999,14.88,5.19,16.93,23.16,3.060000,14.530000,101352.078125,18.450001,34.540001,28.309999,98644.968750,-2.560000,28.420000,28.299999,28.549999,1535.520020,5899.660156,31.660000,200.0,20.570000,15.140000,-99.889999,-208.229996,18.670000,43.650002,-3.700000,-65.019997,628.659973,130.789993,83.690002,-79.660004,-28.030001,-109.809998,-71.989998,35.849998,-17.340000,19.480000,-2277.719971,410.100006,-2321.020020,1064.979980,-816.000000,77.169998,90.349998,-160.020004,413.910004,352.200012,-7267.970215,1100.680054,-3189.610107,1410.839966,-1868.949951,-1254.449951,-602.969971,-72427.679688,-16054.099609,10487.610352,-4560.339844,7128.129883,-2281.449951,-6076.149902,-2209.629883,3864.179932,-3051.209961,-25749.699219,-5160.589844,-1507.910034,3391.320068,-288.519989,1544.020020,944.729980,-1267.750000,0
1,228.899994,29.66,31244.779297,28.719999,82.559998,38.049999,14.37,11.60,17.629999,14.88,5.19,16.93,23.16,3.060000,14.530000,101396.023438,18.450001,34.540001,28.309999,98686.796875,-2.390000,28.420000,28.299999,28.549999,1538.000000,5901.029785,31.660000,200.0,56.540001,-2.390000,-113.059998,-206.979996,23.889999,45.290001,3.630000,-50.560001,615.580017,135.479996,85.250000,-68.430000,-21.219999,-93.940002,-62.790001,28.760000,-7.750000,25.379999,-2287.340088,354.170013,-2270.790039,977.450012,-670.750000,77.480003,-48.070000,-71.800003,507.959991,350.959991,-7189.770020,970.539978,-3095.669922,1382.869995,-1740.290039,-953.140015,-849.109985,-70659.218750,-16485.029297,12437.040039,-5318.370117,8109.370117,-1957.359985,-6672.229980,-3786.459961,2626.550049,-3623.290039,-25474.369141,-5356.700195,-1367.760010,3188.989990,-221.059998,1256.479980,2018.619995,-1110.560059,0
2,220.690002,29.66,31239.269531,28.719999,83.290001,38.049999,14.37,11.60,17.629999,14.88,5.19,16.93,23.16,3.060000,14.530000,101429.250000,18.450001,34.540001,28.309999,98712.851562,-2.760000,28.420000,28.299999,28.549999,1540.319946,5902.180176,31.660000,200.0,94.190002,-20.020000,-118.370003,-194.539993,28.530001,42.799999,9.960000,-32.910000,602.140015,142.059998,86.129997,-61.320000,-12.640000,-82.93

In [54]:
new_test = test_dropped[lasso_selected_col]
new_test.shape

(31354, 82)

In [55]:
y

0         28.744480
1         28.370584
2         28.133059
3         28.256798
4         28.372353
            ...    
375729    17.150955
375730    16.962051
375731    16.915474
375732    16.536760
375733    15.910995
Name: contest-tmp2m-14d__tmp2m, Length: 375734, dtype: float32

In [57]:
#def catboost_mse(iterations, learning_rate, depth, l2_leaf_reg,
#                 random_strength, border_count,
#                 bagging_temperature):
#    model = catboost.CatBoostRegressor(iterations=int(iterations),
#                                       learning_rate=learning_rate,
#                                       depth=int(depth),
#                                       l2_leaf_reg=l2_leaf_reg,
#                                       random_strength=random_strength,
#                                      border_count=int(border_count),
#                                       bagging_temperature=bagging_temperature)
#    return -np.mean(
#        cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))


# Define the bounds for the Bayesian optimization
#bounds = {
#    'iterations': (50, 500),
#    'learning_rate': (0.01, 0.3),
#    'depth': (5, 10),
#    'l2_leaf_reg': (1, 100),
#    'random_strength': (0.01, 1),
#    'border_count': (1, 255),
#   'bagging_temperature': (0.0, 1.0)
#}

# Perform the Bayesian optimization
#optimizer = BayesianOptimization(catboost_mse, bounds)
#optimizer.maximize(init_points=10, n_iter=30)

In [58]:
# Print the best hyperparameters
#print("Best hyperparameters found: ", optimizer.max)
#max_bo_params = optimizer.max['params']
#max_bo_params

In [61]:
#optimizer.max()
# Best hyperparameter found
max_bo_params = {
        'bagging_temperature': 0.6659832168548175,
        'border_count': 148.47340045003136,
        'depth': 5.39094603181388,
        'iterations': 91.48333241446494,
        'l2_leaf_reg': 79.11730348188014,
        'learning_rate': 0.015584961745152205,
        'random_state': 13.47328923458555,
        'random_strength': 0.4030802972117652
}

In [64]:
cbr = catboost.CatBoostRegressor(bagging_temperature=max_bo_params['bagging_temperature'],
                                 border_count=int(max_bo_params['border_count']),
                                 depth=10,
                                 iterations=10000,
                                 l2_leaf_reg=max_bo_params['l2_leaf_reg'],
                                 learning_rate=max_bo_params['learning_rate'],
                                 random_state=13,
                                 random_strength=max_bo_params['random_strength'],
                                 loss_function='RMSE')

In [65]:
cbr.fit(model_df, y)

0:	learn: 9.7280950	total: 173ms	remaining: 28m 48s
1:	learn: 9.5906681	total: 334ms	remaining: 27m 50s
2:	learn: 9.4555485	total: 501ms	remaining: 27m 48s
3:	learn: 9.3233076	total: 664ms	remaining: 27m 38s
4:	learn: 9.1927402	total: 841ms	remaining: 28m
5:	learn: 9.0641948	total: 1.03s	remaining: 28m 32s
6:	learn: 8.9375689	total: 1.21s	remaining: 28m 46s
7:	learn: 8.8134250	total: 1.38s	remaining: 28m 44s
8:	learn: 8.6909349	total: 1.56s	remaining: 28m 47s
9:	learn: 8.5698010	total: 1.73s	remaining: 28m 48s
10:	learn: 8.4505567	total: 1.9s	remaining: 28m 47s
11:	learn: 8.3347077	total: 2.07s	remaining: 28m 46s
12:	learn: 8.2188963	total: 2.27s	remaining: 29m 4s
13:	learn: 8.1066431	total: 2.45s	remaining: 29m 10s
14:	learn: 7.9949191	total: 2.63s	remaining: 29m 9s
15:	learn: 7.8851077	total: 2.8s	remaining: 29m 5s
16:	learn: 7.7765772	total: 2.97s	remaining: 29m 2s
17:	learn: 7.6709469	total: 3.14s	remaining: 28m 59s
18:	learn: 7.5663977	total: 3.31s	remaining: 29m
19:	learn: 7.4646

156:	learn: 1.9289150	total: 27.4s	remaining: 28m 37s
157:	learn: 1.9187572	total: 27.6s	remaining: 28m 37s
158:	learn: 1.9085353	total: 27.8s	remaining: 28m 37s
159:	learn: 1.8987047	total: 27.9s	remaining: 28m 37s
160:	learn: 1.8891509	total: 28.1s	remaining: 28m 36s
161:	learn: 1.8792952	total: 28.3s	remaining: 28m 36s
162:	learn: 1.8699436	total: 28.4s	remaining: 28m 36s
163:	learn: 1.8606195	total: 28.6s	remaining: 28m 36s
164:	learn: 1.8514331	total: 28.8s	remaining: 28m 37s
165:	learn: 1.8426390	total: 29s	remaining: 28m 37s
166:	learn: 1.8339869	total: 29.2s	remaining: 28m 38s
167:	learn: 1.8253748	total: 29.4s	remaining: 28m 38s
168:	learn: 1.8165746	total: 29.5s	remaining: 28m 38s
169:	learn: 1.8081914	total: 29.7s	remaining: 28m 38s
170:	learn: 1.7994388	total: 29.9s	remaining: 28m 38s
171:	learn: 1.7916559	total: 30.1s	remaining: 28m 38s
172:	learn: 1.7837763	total: 30.2s	remaining: 28m 38s
173:	learn: 1.7759517	total: 30.4s	remaining: 28m 38s
174:	learn: 1.7684146	total: 3

309:	learn: 1.2924027	total: 55.5s	remaining: 28m 56s
310:	learn: 1.2902774	total: 55.7s	remaining: 28m 55s
311:	learn: 1.2887174	total: 55.9s	remaining: 28m 55s
312:	learn: 1.2870210	total: 56.1s	remaining: 28m 55s
313:	learn: 1.2852291	total: 56.3s	remaining: 28m 55s
314:	learn: 1.2838071	total: 56.4s	remaining: 28m 55s
315:	learn: 1.2820245	total: 56.6s	remaining: 28m 55s
316:	learn: 1.2800415	total: 56.8s	remaining: 28m 55s
317:	learn: 1.2782235	total: 57s	remaining: 28m 55s
318:	learn: 1.2769941	total: 57.2s	remaining: 28m 54s
319:	learn: 1.2757391	total: 57.3s	remaining: 28m 54s
320:	learn: 1.2743459	total: 57.5s	remaining: 28m 54s
321:	learn: 1.2730312	total: 57.7s	remaining: 28m 54s
322:	learn: 1.2717184	total: 57.9s	remaining: 28m 53s
323:	learn: 1.2702832	total: 58.1s	remaining: 28m 53s
324:	learn: 1.2684225	total: 58.2s	remaining: 28m 53s
325:	learn: 1.2669632	total: 58.4s	remaining: 28m 53s
326:	learn: 1.2653456	total: 58.6s	remaining: 28m 53s
327:	learn: 1.2638224	total: 5

460:	learn: 1.1126033	total: 1m 23s	remaining: 28m 44s
461:	learn: 1.1116557	total: 1m 23s	remaining: 28m 44s
462:	learn: 1.1109372	total: 1m 23s	remaining: 28m 44s
463:	learn: 1.1099310	total: 1m 23s	remaining: 28m 44s
464:	learn: 1.1087213	total: 1m 24s	remaining: 28m 44s
465:	learn: 1.1077772	total: 1m 24s	remaining: 28m 43s
466:	learn: 1.1070647	total: 1m 24s	remaining: 28m 43s
467:	learn: 1.1059544	total: 1m 24s	remaining: 28m 43s
468:	learn: 1.1049674	total: 1m 24s	remaining: 28m 43s
469:	learn: 1.1040812	total: 1m 24s	remaining: 28m 43s
470:	learn: 1.1031044	total: 1m 25s	remaining: 28m 43s
471:	learn: 1.1021160	total: 1m 25s	remaining: 28m 43s
472:	learn: 1.1009172	total: 1m 25s	remaining: 28m 43s
473:	learn: 1.1001271	total: 1m 25s	remaining: 28m 43s
474:	learn: 1.0990782	total: 1m 25s	remaining: 28m 43s
475:	learn: 1.0981362	total: 1m 26s	remaining: 28m 43s
476:	learn: 1.0971252	total: 1m 26s	remaining: 28m 43s
477:	learn: 1.0963405	total: 1m 26s	remaining: 28m 44s
478:	learn

609:	learn: 0.9923793	total: 1m 50s	remaining: 28m 24s
610:	learn: 0.9917058	total: 1m 50s	remaining: 28m 23s
611:	learn: 0.9911537	total: 1m 51s	remaining: 28m 23s
612:	learn: 0.9904976	total: 1m 51s	remaining: 28m 23s
613:	learn: 0.9898689	total: 1m 51s	remaining: 28m 23s
614:	learn: 0.9891196	total: 1m 51s	remaining: 28m 22s
615:	learn: 0.9884144	total: 1m 51s	remaining: 28m 22s
616:	learn: 0.9879272	total: 1m 51s	remaining: 28m 22s
617:	learn: 0.9873122	total: 1m 52s	remaining: 28m 22s
618:	learn: 0.9868171	total: 1m 52s	remaining: 28m 22s
619:	learn: 0.9861832	total: 1m 52s	remaining: 28m 22s
620:	learn: 0.9855698	total: 1m 52s	remaining: 28m 22s
621:	learn: 0.9850630	total: 1m 52s	remaining: 28m 22s
622:	learn: 0.9845270	total: 1m 53s	remaining: 28m 21s
623:	learn: 0.9839248	total: 1m 53s	remaining: 28m 21s
624:	learn: 0.9833789	total: 1m 53s	remaining: 28m 21s
625:	learn: 0.9825525	total: 1m 53s	remaining: 28m 21s
626:	learn: 0.9820241	total: 1m 53s	remaining: 28m 21s
627:	learn

...

9307:	learn: 0.2890165	total: 28m 36s	remaining: 2m 7s
9308:	learn: 0.2889953	total: 28m 36s	remaining: 2m 7s
9309:	learn: 0.2889819	total: 28m 37s	remaining: 2m 7s
9310:	learn: 0.2889645	total: 28m 37s	remaining: 2m 7s
9311:	learn: 0.2889467	total: 28m 37s	remaining: 2m 6s
9312:	learn: 0.2889298	total: 28m 37s	remaining: 2m 6s
9313:	learn: 0.2889107	total: 28m 37s	remaining: 2m 6s
9314:	learn: 0.2888925	total: 28m 37s	remaining: 2m 6s
9315:	learn: 0.2888836	total: 28m 38s	remaining: 2m 6s
9316:	learn: 0.2888665	total: 28m 38s	remaining: 2m 5s
9317:	learn: 0.2888478	total: 28m 38s	remaining: 2m 5s
9318:	learn: 0.2888344	total: 28m 38s	remaining: 2m 5s
9319:	learn: 0.2888202	total: 28m 38s	remaining: 2m 5s
9320:	learn: 0.2888083	total: 28m 39s	remaining: 2m 5s
9321:	learn: 0.2887804	total: 28m 39s	remaining: 2m 5s
9322:	learn: 0.2887614	total: 28m 39s	remaining: 2m 4s
9323:	learn: 0.2887407	total: 28m 39s	remaining: 2m 4s
9324:	learn: 0.2887151	total: 28m 39s	remaining: 2m 4s
9325:	lear

9457:	learn: 0.2865129	total: 29m 4s	remaining: 1m 39s
9458:	learn: 0.2864944	total: 29m 4s	remaining: 1m 39s
9459:	learn: 0.2864810	total: 29m 4s	remaining: 1m 39s
9460:	learn: 0.2864714	total: 29m 5s	remaining: 1m 39s
9461:	learn: 0.2864579	total: 29m 5s	remaining: 1m 39s
9462:	learn: 0.2864309	total: 29m 5s	remaining: 1m 39s
9463:	learn: 0.2864160	total: 29m 5s	remaining: 1m 38s
9464:	learn: 0.2864015	total: 29m 5s	remaining: 1m 38s
9465:	learn: 0.2863864	total: 29m 6s	remaining: 1m 38s
9466:	learn: 0.2863753	total: 29m 6s	remaining: 1m 38s
9467:	learn: 0.2863551	total: 29m 6s	remaining: 1m 38s
9468:	learn: 0.2863382	total: 29m 6s	remaining: 1m 37s
9469:	learn: 0.2863151	total: 29m 6s	remaining: 1m 37s
9470:	learn: 0.2862973	total: 29m 6s	remaining: 1m 37s
9471:	learn: 0.2862828	total: 29m 7s	remaining: 1m 37s
9472:	learn: 0.2862661	total: 29m 7s	remaining: 1m 37s
9473:	learn: 0.2862482	total: 29m 7s	remaining: 1m 37s
9474:	learn: 0.2862354	total: 29m 7s	remaining: 1m 36s
9475:	lear

9605:	learn: 0.2841706	total: 29m 32s	remaining: 1m 12s
9606:	learn: 0.2841541	total: 29m 32s	remaining: 1m 12s
9607:	learn: 0.2841427	total: 29m 32s	remaining: 1m 12s
9608:	learn: 0.2841271	total: 29m 32s	remaining: 1m 12s
9609:	learn: 0.2841131	total: 29m 32s	remaining: 1m 11s
9610:	learn: 0.2840985	total: 29m 32s	remaining: 1m 11s
9611:	learn: 0.2840927	total: 29m 33s	remaining: 1m 11s
9612:	learn: 0.2840768	total: 29m 33s	remaining: 1m 11s
9613:	learn: 0.2840648	total: 29m 33s	remaining: 1m 11s
9614:	learn: 0.2840541	total: 29m 33s	remaining: 1m 11s
9615:	learn: 0.2840356	total: 29m 33s	remaining: 1m 10s
9616:	learn: 0.2840180	total: 29m 34s	remaining: 1m 10s
9617:	learn: 0.2840000	total: 29m 34s	remaining: 1m 10s
9618:	learn: 0.2839833	total: 29m 34s	remaining: 1m 10s
9619:	learn: 0.2839738	total: 29m 34s	remaining: 1m 10s
9620:	learn: 0.2839658	total: 29m 34s	remaining: 1m 9s
9621:	learn: 0.2839488	total: 29m 35s	remaining: 1m 9s
9622:	learn: 0.2839330	total: 29m 35s	remaining: 1

9755:	learn: 0.2818746	total: 29m 59s	remaining: 45s
9756:	learn: 0.2818636	total: 30m	remaining: 44.8s
9757:	learn: 0.2818536	total: 30m	remaining: 44.6s
9758:	learn: 0.2818354	total: 30m	remaining: 44.5s
9759:	learn: 0.2818142	total: 30m	remaining: 44.3s
9760:	learn: 0.2818044	total: 30m	remaining: 44.1s
9761:	learn: 0.2817820	total: 30m	remaining: 43.9s
9762:	learn: 0.2817670	total: 30m 1s	remaining: 43.7s
9763:	learn: 0.2817567	total: 30m 1s	remaining: 43.5s
9764:	learn: 0.2817439	total: 30m 1s	remaining: 43.4s
9765:	learn: 0.2817279	total: 30m 1s	remaining: 43.2s
9766:	learn: 0.2817146	total: 30m 1s	remaining: 43s
9767:	learn: 0.2817015	total: 30m 2s	remaining: 42.8s
9768:	learn: 0.2816865	total: 30m 2s	remaining: 42.6s
9769:	learn: 0.2816691	total: 30m 2s	remaining: 42.4s
9770:	learn: 0.2816577	total: 30m 2s	remaining: 42.2s
9771:	learn: 0.2816464	total: 30m 2s	remaining: 42.1s
9772:	learn: 0.2816320	total: 30m 2s	remaining: 41.9s
9773:	learn: 0.2816155	total: 30m 3s	remaining: 4

9907:	learn: 0.2795624	total: 30m 27s	remaining: 17s
9908:	learn: 0.2795436	total: 30m 27s	remaining: 16.8s
9909:	learn: 0.2795208	total: 30m 28s	remaining: 16.6s
9910:	learn: 0.2795061	total: 30m 28s	remaining: 16.4s
9911:	learn: 0.2794915	total: 30m 28s	remaining: 16.2s
9912:	learn: 0.2794765	total: 30m 28s	remaining: 16s
9913:	learn: 0.2794658	total: 30m 28s	remaining: 15.9s
9914:	learn: 0.2794500	total: 30m 28s	remaining: 15.7s
9915:	learn: 0.2794382	total: 30m 29s	remaining: 15.5s
9916:	learn: 0.2794265	total: 30m 29s	remaining: 15.3s
9917:	learn: 0.2794135	total: 30m 29s	remaining: 15.1s
9918:	learn: 0.2794041	total: 30m 29s	remaining: 14.9s
9919:	learn: 0.2793907	total: 30m 29s	remaining: 14.8s
9920:	learn: 0.2793753	total: 30m 30s	remaining: 14.6s
9921:	learn: 0.2793544	total: 30m 30s	remaining: 14.4s
9922:	learn: 0.2793357	total: 30m 30s	remaining: 14.2s
9923:	learn: 0.2793145	total: 30m 30s	remaining: 14s
9924:	learn: 0.2793041	total: 30m 30s	remaining: 13.8s
9925:	learn: 0.2

In [66]:
predictions = cbr.predict(new_test)

In [67]:
predictions

array([29.21885292, 29.26659527, 29.18086863, ...,  3.6832513 ,
        3.63713151,  3.88230615])

In [68]:
submission = test
submission[target] = predictions 

In [71]:
submission[target]

0        29.218853
1        29.266595
2        29.180869
3        29.126501
4        29.174066
           ...    
31349     4.342244
31350     4.056691
31351     3.683251
31352     3.637132
31353     3.882306
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64

In [70]:
submission[[target, "index"]].to_csv('submission.csv', index = False)

## Ensemble modelling

In [73]:
lgbm_pred = pd.read_csv('y_pred_lgb.csv')

In [86]:
final_pred = submission[target] * 0.3 + 0.7 * lgbm_pred[target]
en_submission = test
en_submission[target] = final_pred
en_submission[[target, "index"]].to_csv('en_submission.csv', index = False)

In [87]:
final_pred

0        28.823403
1        28.693547
2        28.762718
3        28.801255
4        28.735271
           ...    
31349     6.476910
31350     6.313445
31351     5.537076
31352     6.006163
31353     6.275723
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64

In [88]:
en_submission.shape[0]

31354